## Import Data

In [1]:
#list the data
!ls Data

averages.csv
fivethirtyeight_partisan_lean_DISTRICTS.csv
fivethirtyeight_partisan_lean_STATES.csv
vote_predictions.csv


In [2]:
!ls

Capstone Project Proposal - 2.pdf  Data Cleaning.ipynb	   README.md
Data				   PROPUBLICA_API_KEY.TXT


### Get API working

In [3]:
#Load ProPublica API key without exposing it to github. Request one yourself. They're free.
APIfile = open('./PROPUBLICA_API_KEY.TXT',)
APIkey = APIfile.read().replace('\n', '')
print(type(APIkey))
#print(APIkey)

<class 'str'>


In [130]:
#API request function
import requests

#create API key
headers = {'X-API-Key': APIkey}

#load status codes from API documentation(https://projects.propublica.org/api-docs/congress-api/#errors) into dictionary
status_codes = {
    200:'Successful Connection!',
    400:'Bad Request – Your request is improperly formed',
    403:'Forbidden – Your request did not include an authorization header',
    404:'Not Found – The specified record(s) could not be found',
    406:'Not Acceptable – You requested a format that isn’t json or xml',
    500:'Internal Server Error – We had a problem with our server. Try again later',
    503:'Service Unavailable – The service is currently not working. Please try again later'
    }

#load base url and test endpoint
url = 'https://api.propublica.org/congress/v1'
test_endpoint = '/members/new.json'

def API_Request(end_point, verbose=True):
#send a get request to restful API, print status and return response as a dictionary
    response = requests.get(url+end_point, headers=headers)
    #print the response status
    if verbose == True:
        try:
            print(status_codes[response.status_code])
        except: 
            print('API Failure, unknown status code: '+response.status_code)
 
    return response.json()

#test that the API is working. 
new_members = API_Request(test_endpoint)
print(members.keys()) # When the API works this should give the list of attributes per member
print('status:', new_members['status'])
print('no. results:', new_members['results'][0]['num_results'])

Successful Connection!
dict_keys(['status', 'copyright', 'results'])
status: OK
no. results: 20


In [55]:
#print what type of data we can get for each new member
new_members_data = new_members['results'][0]['members']
print(new_members_data[0].keys())


dict_keys(['id', 'api_uri', 'first_name', 'middle_name', 'last_name', 'suffix', 'party', 'chamber', 'state', 'start_date'])


In [56]:
#list some data for 20 members:
for member in new_members_data:
    print(member['first_name'], member['last_name']+':', member['party'])

Rick Scott: R
Michael San Nicolas: D
Jared Golden: D
Martha McSally: R
Mitt Romney: R
Joshua Hawley: R
Mike Braun: R
Carol Miller: R
Bryan Steil: R
Kim Schrier: D
Jennifer Wexton: D
Abigail Spanberger: D
Ben Cline: R
Denver Riggleman: R
Elaine Luria: D
Ben McAdams: D
Colin Allred: D
Sylvia Garcia: D
Chip Roy: R
Veronica Escobar: D


### The API is working.  Let's import voting data

In [70]:
#import test rollcal vote for the senate
recent_endpoint = '/senate/votes/recent.json'
senate_vote_recent = API_Request(vote_endpoint)
print('no. results', senate_vote_test['results']['num_results'])

Successful Connection!
no. results 20


In [73]:
last_20_votes = senate_vote_recent['results']['votes']
print(last_20_votes[0].keys())

dict_keys(['congress', 'chamber', 'session', 'roll_call', 'source', 'url', 'vote_uri', 'bill', 'amendment', 'question', 'question_text', 'description', 'vote_type', 'date', 'time', 'result', 'tie_breaker', 'tie_breaker_vote', 'document_number', 'document_title', 'democratic', 'republican', 'independent', 'total'])


In [91]:
for vote in last_20_votes:
    print(vote['chamber'], vote['congress'], str(vote['roll_call'])+':', vote['description']+'\n')
    

Senate 116 94: A joint resolution to direct the removal of United States Armed Forces from hostilities in the Republic of Yemen that have not been authorized by Congress.

Senate 116 93: Joshua Wolson, of Pennsylvania, to be United States District Judge for the Eastern District of Pennsylvania

Senate 116 92: Raul M. Arias-Marxuach, of Puerto Rico, to be United States District Judge for the District of Puerto Rico

Senate 116 91: Rodolfo Armando Ruiz II, of Florida, to be United States District Judge for the Southern District of Florida

Senate 116 90: Joshua Wolson, of Pennsylvania, to be United States District Judge for the Eastern District of Pennsylvania

Senate 116 89: Raul M. Arias-Marxuach, of Puerto Rico, to be United States District Judge for the District of Puerto Rico

Senate 116 88: Rodolfo Armando Ruiz II, of Florida, to be United States District Judge for the Southern District of Florida

Senate 116 87: Andrew Lynn Brasher, of Alabama, to be United States District Judge f

In [108]:
import pandas as pd
last_20df = pd.DataFrame(last_20_votes)
last_20df.loc[:5,'bill':'description']

,bill,chamber,congress,date,democratic,description
0,"{'bill_id': 'sjres7-116', 'number': 'S.J.RES.7...",Senate,116,2019-05-02,"{'yes': 44, 'no': 0, 'present': 0, 'not_voting...",A joint resolution to direct the removal of Un...
1,{},Senate,116,2019-05-02,"{'yes': 11, 'no': 32, 'present': 0, 'not_votin...","Joshua Wolson, of Pennsylvania, to be United S..."
2,{},Senate,116,2019-05-02,"{'yes': 41, 'no': 2, 'present': 0, 'not_voting...","Raul M. Arias-Marxuach, of Puerto Rico, to be ..."
3,{},Senate,116,2019-05-02,"{'yes': 36, 'no': 7, 'present': 0, 'not_voting...","Rodolfo Armando Ruiz II, of Florida, to be Uni..."
4,{},Senate,116,2019-05-01,"{'yes': 11, 'no': 34, 'present': 0, 'not_votin...","Joshua Wolson, of Pennsylvania, to be United S..."
5,{},Senate,116,2019-05-01,"{'yes': 41, 'no': 4, 'present': 0, 'not_voting...","Raul M. Arias-Marxuach, of Puerto Rico, to be ..."


In [131]:
#use above to test get voting record for the latest bill
congress = last_20df.loc[0,'congress']
chamber = last_20df.loc[0,'chamber']
session = last_20df.loc[0,'session']
roll_call = last_20df.loc[0,'roll_call']

lv_endpoint = '/'+str(congress)+'/'+chamber.lower()+'/sessions/'+str(session)+'/votes/'+str(roll_call)+'.json'
print('using endpoint:', lv_endpoint)

latest_vote = API_Request(lv_endpoint)

using endpoint: /116/senate/sessions/1/votes/94.json
Successful Connection!


In [132]:
vote = latest_vote['results']['votes']['vote']
vote.keys()

dict_keys(['congress', 'session', 'chamber', 'roll_call', 'source', 'url', 'bill', 'amendment', 'question', 'question_text', 'description', 'vote_type', 'date', 'time', 'result', 'tie_breaker', 'tie_breaker_vote', 'document_number', 'document_title', 'democratic', 'republican', 'independent', 'total', 'positions'])

In [140]:
#load a dataframe of everyone's position on the latest vote:
latest_vote_positions_df = pd.DataFrame(vote['positions'])
latest_vote_positions_df

,dw_nominate,member_id,name,party,state,vote_position
0,0.323,A000360,Lamar Alexander,R,TN,No
1,-0.521,B001230,Tammy Baldwin,D,WI,Yes
2,0.534,B001261,John Barrasso,R,WY,No
3,-0.208,B001267,Michael Bennet,D,CO,Not Voting
4,0.614,B001243,Marsha Blackburn,R,TN,No
5,-0.411,B001277,Richard Blumenthal,D,CT,Yes
6,0.429,B000575,Roy Blunt,R,MO,No
7,-0.618,B001288,Cory Booker,D,NJ,Yes
8,0.401,B001236,John Boozman,R,AR,No
9,NaN,B001310,Mike Braun,R,IN,No


In [143]:
#Yeah, that's right! That's vote data!
#let's add some more:
print('Importing', end='')
for call in range(roll_call-1,0,-1):
    call_endpoint = '/'+str(congress)+'/'+chamber.lower()+'/sessions/'+str(session)+'/votes/'+str(call)+'.json'
    call_response = API_Request(call_endpoint, verbose=False)
    call_vote = call_response['results']['votes']['vote']
    call_position = pd.DataFrame(call_vote['positions'])
    latest_vote_positions_df = pd.merge(latest_vote_positions_df, call_position[['member_id','vote_position']], on='member_id', suffixes=('','_'+str(call))) 
    print('.', end='')
    
print('All Senate Votes for the current session:')
latest_vote_positions_df

Importing.............................................................................................All Senate Votes for the current session:


,dw_nominate,member_id,name,party,state,vote_position,vote_position_93,vote_position_92,vote_position_91,vote_position_90,...,vote_position_10,vote_position_9,vote_position_8,vote_position_7,vote_position_6,vote_position_5,vote_position_4,vote_position_3,vote_position_2,vote_position_1
0,0.323,A000360,Lamar Alexander,R,TN,No,Yes,Yes,Yes,Yes,...,Yes,Yes,No,Not Voting,No,No,Yes,Yes,Yes,Yes
1,-0.521,B001230,Tammy Baldwin,D,WI,Yes,No,Yes,Yes,No,...,Yes,No,Yes,No,Yes,Yes,No,No,No,No
2,0.534,B001261,John Barrasso,R,WY,No,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes
3,-0.208,B001267,Michael Bennet,D,CO,Not Voting,Not Voting,Not Voting,Not Voting,No,...,Yes,No,Yes,No,Yes,Yes,No,No,No,No
4,0.614,B001243,Marsha Blackburn,R,TN,No,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes
5,-0.411,B001277,Richard Blumenthal,D,CT,Yes,No,Yes,Yes,No,...,Yes,No,Yes,No,Yes,Yes,No,No,No,No
6,0.429,B000575,Roy Blunt,R,MO,No,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes
7,-0.618,B001288,Cory Booker,D,NJ,Yes,Not Voting,Not Voting,Not Voting,No,...,Yes,No,No,No,Yes,Yes,No,No,No,No
8,0.401,B001236,John Boozman,R,AR,No,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes
9,NaN,B001310,Mike Braun,R,IN,No,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes
